In [1]:
import numpy as np
import matplotlib.pyplot as plt

$w(t,c)= \frac{1}{\sqrt{2^{SF}}}e^{2\pi (c \frac{bw}{2^{SF}-1}+\frac{bw^2}{2^{SF}-1}t)t}$ 

donde $(c \frac{bw}{2^{SF}-1}+\frac{bw^2}{2^{SF}-1}t)$ se limita al rango $[0,bw]$ y cuando lo supera se resta $bw$

In [10]:
sf = 7
bw = 125000

def freq_evol(sf,bw,symbol, timeline):
    slope = bw**2/(2**sf - 1)
    y_intercept = symbol*(bw/2**sf)
    freq_evol = []
    for i in range(len(timeline)):
        freq_evol.append(slope*timeline[i] + y_intercept)
        if freq_evol[i] > bw:
            freq_evol[i] = freq_evol[i] - bw
    return freq_evol

def signal_from_evol(sf, bw, freq, timeline):
    signal = []
    coeff = 1/(np.sqrt(2**sf))
    for i in range(len(timeline)):
        phase = 2*np.pi*freq[i]
        signal.append(coeff * np.exp(1j*phase*timeline[i]))
    return signal

def generate_downchirp(sf,bw, timeline):
    freq = []
    signal = []
    coeff = 1/(np.sqrt(2**sf))
    for i in range(len(timeline)):
        freq.append( bw - ( bw**2 / (2**sf - 1) ) * timeline[i] )
        phase = 2*np.pi*freq[i]*timeline[i]
        signal.append(coeff * np.exp(1j * phase))
    return freq, signal

def demodulate(downchirp, signal):
    demod = []
    for i in range(len(timeline)):
        demod.append(downchirp[i]*signal[i])
    symbol= np.argmax(np.fft.fft(demod))
    return symbol

errors = False
for i in range(0,2**sf):
    timeline = np.linspace(0, (2**sf-1)*(1/bw), 2**sf)
    freq = freq_evol(sf,bw, i,timeline)
    signal = signal_from_evol(sf,bw,freq,timeline)
    downchirp, downchirp_signal = generate_downchirp(sf,bw,timeline)
    demod_symbol = demodulate(downchirp_signal,signal)
    if i != demod_symbol:
        errors = True
        print("Symbol: ", i, "Demodulated unsuccesfully as: ", demod_symbol)
print('Errors found: ', errors)


Errors found:  False
